In [0]:
%pip install databricks-sdk --upgrade

In [0]:
dbutils.library.restartPython()

In [0]:
user_name = spark.sql("select current_user()").collect()[0][0]
scope_name = user_name.split(sep="@")[0].replace(".", "-")
scope_name

In [0]:
dbutils.widgets.text("secret_scope", scope_name, "Databricks Secret Scope")
dbutils.widgets.text("secret_key", "", "Databricks Secret Key")
dbutils.widgets.text("secret_key_value", "", "Secret Key Value")

In [0]:
secret_scope = dbutils.widgets.get("secret_scope")
secret_scope_key = dbutils.widgets.get("secret_key")
secret_key_value = dbutils.widgets.get("secret_key_value")
# multi_line_secret_key_value = """<paste multiple line secret key value here, useful for PEM files>"""
multi_line_secret_key_value = None

In [0]:
from databricks.sdk import WorkspaceClient

w = WorkspaceClient()

In [0]:
scopes = w.secrets.list_scopes()
scopes = [scope.as_dict() for scope in scopes]

In [0]:
scope_exists = any(scope['name'] == secret_scope for scope in scopes)
display(scope_exists)

In [0]:
if scope_exists == True:
  print(f"{secret_scope} secret scope already exists")
else:
  w.secrets.create_scope(scope=secret_scope)

In [0]:
if multi_line_secret_key_value not in [None, '', """<paste multiple line secret key value here, useful for PEM files>"""]: 
  w.secrets.put_secret(scope=secret_scope, key=secret_scope_key, string_value=multi_line_secret_key_value)
else:
  if secret_key_value not in [None, '']:
    w.secrets.put_secret(scope=secret_scope, key=secret_scope_key, string_value=secret_key_value)

In [0]:
dbutils.secrets.get(scope=secret_scope, key=secret_scope_key) in [secret_key_value, multi_line_secret_key_value]

In [0]:
# dbutils.widgets.removeAll()